In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import scipy as sp
import patsy
import pystan

import statsmodels.api as sm

pd.set_option('max_columns', 50)


In [9]:
url = "http://stat.columbia.edu/~gelman/arm/examples/arsenic/wells.dat"
wells = pd.read_table(url, sep='\s+', header=0, index_col=0)
wells["dist100"] = wells['dist']/100

y = wells['switch']
wellsNP = wells.as_matrix()
y_np = wellsNP[:, 0]

a = sp.special.logit(np.mean(y_np))

f = "~ 0 + dist + arsenic"
x = patsy.dmatrix(f, wells, return_type='dataframe')
nrow, ncol = x.shape


data_loo = {'N': nrow , 'P': ncol, 'y': y, 'x': x, 'a': a }


In [3]:

logistic = '''
data {
  int<lower=0> N; 
  int<lower=0> P;
  int<lower=0,upper=1> y[N];
  matrix[N,P] x; 
  real a;
}
parameters {
  real beta0;
  vector[P] beta;
}
model {
  beta0 ~ student_t(7, a, 0.1);
  beta ~ student_t(7, 0, 1);
  y ~ bernoulli_logit(beta0 + x * beta);
}
generated quantities {
  vector[N] log_lik;
  for (n in 1:N)
    log_lik[n] <- bernoulli_logit_log(y[n], beta0 + x[n] * beta);
}
'''

In [4]:
fit1 = pystan.stan(model_code=logistic, data=data_loo, iter=1000, chains=4)



In [5]:
parameter_name = "log_lik"

matrix = fit1.extract(pars=parameter_name)
logLik = matrix['log_lik']


In [7]:
import sys
sys.path.insert(0, '/Users/detcitty/Documents/Stan/loo/loo/')

from psisloo import psisloo



In [8]:
test = psisloo(logLik)
print test

/Users/detcitty/Documents/Stan/loo/loo/gpdfitnew.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif sort == 'in-place':
/Users/detcitty/Documents/Stan/loo/loo/gpdfitnew.py:92: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  bs /= 3 * x[sort[np.floor(n/4 + 0.5) - 1]]


(-1969.0019513205257, array([-0.32823959, -0.71973467, -1.16089959, ..., -0.81852037,
       -0.77100823, -0.60962946]), array([-0.04715097, -0.13412989,  0.02238858, ..., -0.2691381 ,
       -0.25719099, -0.12838326]))
